<a href="https://colab.research.google.com/github/youngolax/Olato_B30927/blob/main/Sample_test_cases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **A SAMPLE TEST CASE FOR EACH TEST TOOL**

**Sample case for Unit Test**

In [ ]:
import pytest
import requests
from unittest.mock import patch

@pytest.fixture
def quiz_data():
    return {
        "answers": [
            {"question_id": 1, "selected": "A", "correct_answer": "A"},
            {"question_id": 2, "selected": "B", "correct_answer": "C"}
        ]
    }

@patch("requests.post")
def test_calculate_score(mock_post, quiz_data):
    # Mock API response
    mock_post.return_value.status_code = 200
    mock_post.return_value.json.return_value = {"score": 50}  # 1/2 correct

    response = requests.post("http://localhost:3000/api/quizzes/1/submit", json=quiz_data)
    assert response.status_code == 200, "Expected 200 OK"
    assert response.json()["score"] == 50, "Score should be 50% for one correct answer"

**Sample for Integration Test**

In [1]:
import pytest
import requests

@pytest.fixture
def auth_token():
    # Simulate login to get JWT
    response = requests.post("http://localhost:3000/api/users/login", json={
        "email": "teacher@example.com",
        "password": "password123"
    })
    assert response.status_code == 200, "Login failed"
    return response.json()["token"]

def test_create_quiz(auth_token):
    quiz_data = {
        "title": "Math Quiz",
        "questions": [
            {
                "text": "What is 1+1?",
                "type": "multiple-choice",
                "options": ["2", "3", "4", "5"],
                "correct_answer": "2"
            }
        ]
    }
    headers = {"Authorization": f"Bearer {auth_token}"}
    response = requests.post("http://localhost:3000/api/quizzes", json=quiz_data, headers=headers)

    assert response.status_code == 201, f"Expected 201 Created, got {response.status_code}"
    assert response.json()["title"] == "Math Quiz", "Quiz title mismatch"
    assert len(response.json()["questions"]) == 1, "Expected one question"
    assert response.json()["questions"][0]["text"] == "What is 1+1?", "Question text mismatch"

Sample case for System test (Cypress)

In [3]:
!pip install playwright

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 MB 13.9 MB/s eta 0:00:00


In [4]:
import pytest
from playwright.sync_api import sync_playwright

@pytest.fixture(scope="module")
def browser_context():
    with sync_playwright() as p:
        browser = p.chromium.launch(headless=True)
        context = browser.new_context()
        yield context
        context.close()
        browser.close()

def test_take_quiz(browser_context):
    page = browser_context.new_page()

    # Login
    page.goto("http://localhost:3000/login")
    page.fill('input[name="email"]', "student@example.com")
    page.fill('input[name="password"]', "password123")
    page.click('button[type="submit"]')
    assert page.url.endswith("/dashboard"), "Login failed"

    # Navigate to quiz
    page.goto("http://localhost:3000/quizzes/1")
    page.select_option('select[name="question-1"]', "A")  # Assume correct answer
    page.click('button[data-testid="submit-quiz"]')

    # Verify score
    score_text = page.text_content('[data-testid="score"]')
    assert "Your score: 100%" in score_text, "Expected full score for correct answer"